In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-23 15:28:24.655360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745422104.670439 3563266 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745422104.675116 3563266 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745422104.688354 3563266 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, os, peft, pickle, pytest, ipytest, pd, transformers, torch, time, threading, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, csv, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, tsv_headers, genome_list, temp_list, i, tsvfile, tsv_writer, search_with_strings, map_data, print_trainable_parameters, metric, initialize_heartbeat, heartbeat, end_heartbeat, comp_metrics_output, compute_metrics, make_trainer, get_dat

In [2]:
mistral, mistral_token=model_from_pkl("Mistral-7B-Instruct-v0.1")

In [3]:
llama, llama_token=model_from_pkl("Llama-3.2-1B-Instruct")

In [4]:
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": """You are a helpful assistant, 
        if I ask you a question you can't answer, please reply 'I can't answer that' """}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=256)[0]['generated_text'][-1]
    print(answer)
    base_pairs=""
    content=""
    base_pairs=get_pairs(answer) # run the get_pairs method to return just the genome data from a response
    messages.append(answer)
    return pipe, messages, base_pairs

In [5]:
def get_pairs(output):
    base_pairs=""
    content=""
    if ":" in output['content']: #the base pairs are output after a colon
        content=output['content'].split(":")[1] # take all of the string after the colon
        for i in range(len(content)):
            if content[i]=='A' or content[i]=='C' or content[i]=='T' or content[i]=='G': # get the first base pair character
                base_pairs=content[i:] # take the rest of the following pairs
                break # stop the loop when the base pairs are found
        return base_pairs

In [6]:
def prompt_with_data(model, tokenizer, genome, pairs):
    pipe=pipeline("text-generation", model=model, tokenizer=tokenizer)
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What base pairs are present in genome " + genome + "?"},
        {"role": "assistant", "content": "The starting base pairs in " + genome + " are:" + pairs}
    ]
    
    output=pipe(messages, max_new_tokens=512)[0]
    print(output)
    base_pairs=output['generated_text'][-1]['content'].split(":")[1] # extract just the generated pairs
    base_pairs=base_pairs.split(pairs, 1)[1] # split after the base pairs that were fed to the model
    return base_pairs

In [7]:
# prompting the base Mistral model with the same prompts as the fine-tuned ones
model, answer, pairs=continued_convo(mistral, mistral_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Sure, here are ten example base pairs for a genome:\n\n1. A-T\n2. C-G\n3. G-C\n4. T-A\n5. A-C\n6. G-T\n7. C-A\n8. T-G\n9. A-G\n10. G-A\n\nThese are the four nucleotide bases that make up DNA, and they are paired up in a specific way to form the genetic code.'}


In [9]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=mistral, tokenizer=mistral_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGACT"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome: CGACTGACGAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCT

In [10]:
search_with_strings(ds_gst1_train, pairs)

('CGACTGACGAGC', 12, 11)

In [12]:
output=prompt_with_data(mistral, mistral_token, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LIXJ01000001.1 are:CGACTCAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCT

('AGCTAGCTAGCT', 12, 3)

In [15]:
output=prompt_with_data(mistral, mistral_token, "JAHFAU010000168.11", "CTCCTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.11?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000168.11 are:CTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTC

('CTCCTCCTCCTCCTCC', 16, 2)

In [16]:
# see how much test data the model can extract
output=prompt_with_data(mistral, mistral_token, "JAHFAU010000093.1", "CGATC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000093.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000093.1 are:CGATCAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA

('AGCTAGCTAGCT', 12, 3)

In [18]:
model, answer, pairs=continued_convo(mistral, mistral_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': " I can't answer that as the first 20 base pairs of a genome can vary greatly depending on the specific organism and species."}


In [ ]:
# the prompt above produces 15-17 character strings of training data with the fine-tuned model, and the base model won't even answer it

In [33]:
output=prompt_with_data(mistral, mistral_token, "IMGPR_plasmid_3300006450_000026", "CAAAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_3300006450_000026?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_3300006450_000026 are:CAAAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGAGA

('AGAGAGAGAGAGAGA', 15, 1)

In [28]:
output=prompt_with_data(mistral, mistral_token, "NZ_LSRP01000058.1", "GTAGA")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LSRP01000058.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LSRP01000058.1 are:GTAGAAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA

('AGCTAGCTAGCT', 12, 3)

In [30]:
output=prompt_with_data(mistral, mistral_token, "CAKUCL010000105.1", "GCCAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome CAKUCL010000105.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in CAKUCL010000105.1 are:GCCAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC

('CTAGCTAGCTAG', 12, 1)

In [35]:
output=prompt_with_data(mistral, mistral_token, "IMGPR_plasmid_2698536519_000009", "CAGCA")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_2698536519_000009?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_2698536519_000009 are:CAGCAAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA

('AGCTAGCTAGC', 11, 1)

In [ ]:
# now try these prompts with the base llama3 model

In [36]:
model, answer, pairs=continued_convo(llama, llama_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are ten example base pairs for a genome:\n\n1. A (Adenine)\n2. T (Thymine)\n3. C (Cytosine)\n4. G (Guanine)\n5. A (Adenine)\n6. C (Cytosine)\n7. G (Guanine)\n8. T (Thymine)\n9. A (Adenine)\n10. C (Cytosine)\n\nThese base pairs are the fundamental building blocks of DNA and are arranged in a specific order to form the genetic code.'}


In [37]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=llama, tokenizer=llama_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGACT"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are twenty example base pairs for a genome: CGACTAGCTAGCTAGCTAGC\n\nIn this example:\n\n* C (cytosine) is represented by 'C'\n* G (guanine) is represented by 'G'\n* A (adenine) is represented by 'A'\n* T (thymine) is represented by 'T'\n\nThis is a simple representation of a short DNA sequence, and actual DNA sequences can be much longer and more complex."}


In [39]:
# the above only elaborated for a small part of the genome and then more or less ignored the prompt
search_with_strings(ds_gst1_train, pairs)

('CGACTAGCTA', 10, 9)

In [40]:
output=prompt_with_data(llama, llama_token, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LIXJ01000001.1 are:CGACTCAGCAGTCAATGATCAGCTGAAATGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGCTATGAAACTTGGAACTTGGTCAAAATGC

('AGCAGTCAATGAT', 13, 3)

In [41]:
output=prompt_with_data(llama, llama_token, "JAHFAU010000168.11", "CTCCTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.11?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000168.11 are:CTCCTCCTCCTCCTTTC.'}]}


('CTCCTCCTTTC', 11, 31)

In [ ]:
# the base llama3 model produces very short strings in general

In [44]:
output=prompt_with_data(llama, llama_token, "JAHFAU010000093.1", "CGATC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000093.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHFAU010000093.1 are:CGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATCAGCAGTACGTAGTACGATC

('AGCAGTACGTAG', 12, 2)

In [48]:
model, answer, pairs=continued_convo(llama, llama_token, """Please provide five examples for what
    the first 20 base pairs of a genome may look like""") # a vague list of examples is provided here, the fine-tuned model produced a long string

Device set to use cuda:0


{'role': 'assistant', 'content': "Here are five examples of what the first 20 base pairs of a genome may look like:\n\n1. **AATCG**: This sequence is a common starting point for many genomes, including the human genome. It's a simple example of a base pair sequence.\n\n2. **ATCGTA**: This sequence is similar to the first one but with the order of the base pairs reversed. It's another common starting point for many genomes.\n\n3. **ACGTCA**: This sequence is another variation of the first two examples. It's also a common starting point for many genomes.\n\n4. **ATCGATC**: This sequence is a simple example of a base pair sequence with an additional nucleotide.\n\n5. **ATCGATCG**: This sequence is a longer example of a base pair sequence with an additional nucleotide."}


In [49]:
search_with_strings(ds_gst1_train, "ATCGATCG")

('ATCGATCG', 8, 1331)

In [53]:
output=prompt_with_data(llama, llama_token, "IMGPR_plasmid_3300006450_000026", "CAAAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_3300006450_000026?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_3300006450_000026 are:CAAAGGCGAAGGCAAAG.'}]}


('GCGAAGGCAAAG', 12, 27)

In [57]:
output=prompt_with_data(llama, llama_token, "NZ_LSRP01000058.1", "GTAGA")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LSRP01000058.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in NZ_LSRP01000058.1 are:GTAGAAGCTGATAGCTATAGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATG

('AGCTGATAGCT', 11, 29)

In [ ]:
# it took multiple attempts for the model to respond to the prompt above with an answer longer than 10 or so characters

In [62]:
output=prompt_with_data(llama, llama_token, "CAKUCL010000105.1", "GCCAG")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome CAKUCL010000105.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in CAKUCL010000105.1 are:GCCAGCCGU.'}]}


('CCG', 3, 7483)

In [63]:
# apart from one attempt, the model continually responds to this prompt with a sequence containing a U

In [69]:
output=prompt_with_data(llama, llama_token, "IMGPR_plasmid_2698536519_000009", "CAGCA")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_2698536519_000009?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_2698536519_000009 are:CAGCAACGAA.'}]}
('ACGAA', 5, 6839)


('ACGAA', 5, 736)

In [71]:
output=prompt_with_data(llama, llama_token, "IMGPR_plasmid_2698536519_000009", "CAGCA")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome IMGPR_plasmid_2698536519_000009?'}, {'role': 'assistant', 'content': 'The starting base pairs in IMGPR_plasmid_2698536519_000009 are:CAGCAATTGATGATCAGC.'}]}
('ATTGATGATCAGC', 13, 5)


('ATTGATGATCAG', 12, 2)

In [ ]:
# second prompt with this question was to see if it would produce longer output

In [ ]:
# overall, the base models have far weaker performance than the fine-tuned ones